In [ ]:
from quantopian.pipeline import Pipeline  
from quantopian.pipeline.data import EquityPricing, factset  
from quantopian.pipeline.domain import US_EQUITIES  
from quantopian.research import run_pipeline
import quantopian.pipeline.factors as Factors
import quantopian.pipeline.filters as Filters
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data import Fundamentals  
from quantopian.pipeline.factors.fundamentals import MarketCap
import numpy as np
import pandas as pd
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus

In [ ]:
def make_pipeline():

    sphere = mstar.asset_classification.morningstar_economy_sphere_code.latest  
    industry_code = mstar.asset_classification.morningstar_industry_code.latest  
    industry_group = mstar.asset_classification.morningstar_industry_group_code.latest  
    sector = mstar.asset_classification.morningstar_sector_code.latest
    
    returns = Factors.Returns(window_length = 2)
    latest_close = EquityPricing.close.latest
    market_cap =  Fundamentals.market_cap.latest
    pe_ratio = mstar.valuation_ratios.pe_ratio.latest
    pb_ratio = mstar.valuation_ratios.pb_ratio.latest
    demeaned_return_within_industry = returns.demean(groupby=industry_group, mask=returns.notnan())
    
    current_eps = Fundamentals.basic_eps_earnings_reports.latest
    forecast_eps = PeriodicConsensus.slice('EPS', 'qf', 4)
    forecast_eps_mean=forecast_eps.mean.latest
    estimated_growth_factor= (forecast_eps_mean - current_eps) / current_eps

    return Pipeline(
        columns={
            'market_cap': market_cap,
            'P/E': pe_ratio,
            'P/B': pb_ratio,
            'price': latest_close,
            'industry_group': industry_group,
            'demeaned_return_within_industry': demeaned_return_within_industry,
            'estimated_growth_factor': estimated_growth_factor
        },
        domain=CN_EQUITIES
    )

my_pipe = make_pipeline()

result = run_pipeline(my_pipe, '2017-06-15','2017-06-15')
result.index.rename(['date', 'stock'], inplace=True)  
oneday = result.loc[('2017-06-15')]

oneday.head()

In [ ]:
averages_by_group = result.groupby(['industry_group']).mean()

merged=result.merge(pd.DataFrame(averages_by_group['P/E']).reset_index(), on='industry_group')
merged1=merged.rename(columns={'P/E_x':'P/E','P/E_y':'industry_P/E'}).set_index(result.index)

merged=merged1.merge(pd.DataFrame(averages_by_group['P/B']).reset_index(), on='industry_group')
screen=merged.rename(columns={'P/B_x':'P/B','P/B_y':'industry_P/B'}).set_index(result.index)

screen

In [ ]:
rank_30=int(np.round(len(screen)*0.3))
top_30=screen.nlargest(rank_30,'market_cap')
top_30

In [ ]:
selected=top_30[(top_30['P/B']<top_30['industry_P/B']) & (top_30['P/E']<top_30['industry_P/E']) & (top_30['estimated_growth_factor']>0)]

In [ ]:
selected

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.research import run_pipeline
quantopian.pipeline.filters.Q500US(minimum_market_cap=500000000)

# Define a 5-day simple moving average sentiment factor.
mean_sentiment_5day = SimpleMovingAverage(inputs=[sentiment.sentiment_signal], window_length=15)

# Add the sentiment factor to a pipeline.
pipe = Pipeline(
    columns={
        'mean_sentiment_5day': mean_sentiment_5day,
    },
    domain=US_EQUITIES,
)

# Run the pipeline for a year and print the first few rows of the result.
df = run_pipeline(pipe, '2020-01-01', '2020-05-05')
print(df.head())

In [ ]:
df.at[symbol('AAPL'),'mean_sentiment_5day']

In [ ]:
from quantopian.research.experimental import query_self_serve_data  
query_self_serve_data(robintrack.columns,'US',assets=symbols(['TSLA'])).tail(20)  